In [319]:
import importlib
import numpy as np
import matplotlib.pyplot as plt
from math import *
def dynamic_import(module_name, function_name):
	module = importlib.import_module(module_name)
	function = getattr(module, function_name)
	return function
calculatePHI_LIQUID_NONESRK = dynamic_import('calculate', 'calculatePHI_LIQUID_NONESRK')
calculatePHI_VAPOR_SRK = dynamic_import('calculate', 'calculatePHI_VAPOR_SRK')
read_excel = dynamic_import('read', 'read_excel')
getTHETA = dynamic_import('read', 'getTHETA')
predict_THETA = dynamic_import('calcute_theta', 'predict_THETA')

In [320]:
features = [
    'Tr', 'Pr', 'LIQUID PL', 'LIQUID K', 'LIQUID MU', 'LIQUID SIGMA',
    'TOTAL RHO', 'LIQUID DHVL', 'M', 'Tc', 'Pc',
    'FREEZEPT', 'TB', 'omiga'
]

In [321]:
pres = read_excel('./DATA/物性系数.xlsx',0,1)
# we test 110-62-3
ct = list(pres.keys())
ik = 6 # 1,2,3,4,5,6
dff = {1:0,
	   2:1,
	   3:5,
	   4:2,
	   5:4,
	   6:3}
TEST_LIST = pres[ct[ik]]
SATlist = TEST_LIST[9:18] # 计算饱和蒸气压经验公式系数
print(SATlist)
print(TEST_LIST)

[233.88, -18507.0, 0, 0, -30.353, 1.3073e-05, 2, 286.65, 708.0]
[128.169, 708.0, 3620000, 0.41, 0.252, 286.65, 501.15, 0.614415, nan, 233.88, -18507.0, 0, 0, -30.353, 1.3073e-05, 2, 286.65, 708.0, nan, 0.21964, -0.00022404, 0, 0, 0, 286.65, 501.15, nan, -12.346, 2046.0, -0.00023475, 0, 0, 286.65, 501.15, nan, 0.067016, 1.2402, 0.0, 0, 0, 286.65, 708.0, nan, 0.61496, 0.25215, 708.0, 0.28571, 0, 286.65, 708.0, nan, 149510000, 1.1546, 0.083403, -0.72697, 0, 286.65, 708.0]


In [322]:
Matterdata = read_excel('./DATA/test_phi.xlsx',1,dff[ik])
print(dff[ik])
KEY_SET = list(Matterdata.keys())
TEMP = Matterdata[KEY_SET[0]][1:] # K
PESS = Matterdata[KEY_SET[1]][1:] # Pa
VOLE = list(np.array(Matterdata[KEY_SET[2]][1:])/(10**6)) # m3/mol
TC = Matterdata[KEY_SET[10]][1:] # K
PC = Matterdata[KEY_SET[11]][1:] # Pa
OMEGA = Matterdata[KEY_SET[16]][1:] # 1
M = Matterdata[KEY_SET[9]][1:] # g/mol
PHI = Matterdata[KEY_SET[18]][1:]
THETALIST = []
for i in range(len(TEMP)):
	thetat = getTHETA(PESS[i],VOLE[i],TEMP[i],TC[i],PC[i],OMEGA[i])
	THETALIST.append(thetat)
TR = list(np.array(TEMP)/np.array(TC))
PR = list(np.array(PESS)/np.array(PC))
DATASET = {
	'Tr':TR,
	'pr':PR,
	KEY_SET[3]:Matterdata[KEY_SET[3]][1:],
	KEY_SET[4]:Matterdata[KEY_SET[4]][1:],
	KEY_SET[5]:Matterdata[KEY_SET[5]][1:],
	KEY_SET[6]:Matterdata[KEY_SET[6]][1:],
	KEY_SET[7]:Matterdata[KEY_SET[7]][1:],
	KEY_SET[8]:Matterdata[KEY_SET[8]][1:],
	'M(g/mol)':M,
	'Tc(K)':TC,
	'pc(Pa)':PC,
	KEY_SET[14]:Matterdata[KEY_SET[14]][1:],
	KEY_SET[15]:Matterdata[KEY_SET[15]][1:],
	'w':OMEGA,
}
print(THETALIST)

3
[np.float64(0.8375858202352884), np.float64(0.8379635277554884), np.float64(0.8383379082521911), np.float64(0.838709012257133), np.float64(0.8390768891695808), np.float64(0.8394415872903324), np.float64(0.8438895605834092), np.float64(0.8443755343799656), np.float64(0.8448566894742173), np.float64(0.8453331097240404), np.float64(0.845804876815772), np.float64(0.8462720703397861), np.float64(0.849611513178963), np.float64(0.8502382656751551), np.float64(0.8508579606716627), np.float64(0.8514707402919557), np.float64(0.852076742366704), np.float64(0.8526761006084224), np.float64(0.8545895104697271), np.float64(0.8554018480775862), np.float64(0.8562036842332309), np.float64(0.8569952665974159), np.float64(0.8577768340063932), np.float64(0.8585486168960187), np.float64(0.8586052498461265), np.float64(0.8596666050062439), np.float64(0.8607119924391582), np.float64(0.861741859658895), np.float64(0.8627566351087526), np.float64(0.863756729257952), np.float64(0.8613529072929124), np.float64(

In [323]:
thetacc = []
KEY_SET2 = list(DATASET.values())
for i in range(len(THETALIST)):
	feature = [c[i] for c in KEY_SET2]
	input_feature = {features[j]:feature[j] for j in range(len(feature))}
	theta_pre = predict_THETA(input_feature)
	thetacc.append(theta_pre)
print(thetacc)
print(input_feature)

[np.float64(0.8821793770902338), np.float64(0.8821793716352979), np.float64(0.8821793661803616), np.float64(0.8821793607254256), np.float64(0.8821793552704895), np.float64(0.8821793498155532), np.float64(0.8768691109015292), np.float64(0.8768691067042311), np.float64(0.8768691025069333), np.float64(0.8768690983096354), np.float64(0.8768690941123374), np.float64(0.8768690899150391), np.float64(0.8531286230482149), np.float64(0.85312861794961), np.float64(0.8531286128510054), np.float64(0.8531286077524003), np.float64(0.8531286026537959), np.float64(0.853128597555191), np.float64(0.860895119019375), np.float64(0.8608951137615575), np.float64(0.8608951085037402), np.float64(0.8608951032459227), np.float64(0.8608950979881054), np.float64(0.8608950927302876), np.float64(1.1476089483705227), np.float64(1.1476089433326049), np.float64(1.1476089382946864), np.float64(1.147608933256769), np.float64(1.1476089282188502), np.float64(1.147608923180932), np.float64(2.03344000685093), np.float64(2.03

In [324]:
# 计算逸度
phi = []
phc = []
for k in range(len(THETALIST)):
	# Phi = calculatePHI_LIQUID_NONESRK(PESS[k],VOLE[k],TEMP[k],TC[k],PC[k],OMEGA[k],thetacc[k],SATlist)
	phi.append("nan")
	Phk = calculatePHI_LIQUID_NONESRK(PESS[k],VOLE[k],TEMP[k],TC[k],PC[k],OMEGA[k],THETALIST[k],SATlist)
	phc.append(Phk)
print(phi)
print(phc)
print(PHI)

['nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan']
[0.0008478022538858506, 0.0003959337451899224, 0.0002632219477091159, 0.00019989599443438072, 0.0001628920309369944, 0.00013867577807142345, 0.004845343490026614, 0.002259952588602624, 0.0015005373912296358, 0.0011380931263068558, 0.0009262400817473247, 0.0007875449919272376, 0.020068097017000172, 0.0093502853147675, 0.006201798933160972, 0.004698890748709857, 0.0038202265519248547, 0.003244816867460102, 0.0646757451454469, 0.030108362378051926, 0.019953028026700004, 0.015104873726819769, 0.012269960953676372, 0.010413055533425971, 0.1709869798343107, 

In [325]:
with open(f'./RES/ccp/test{ct[ik]}.csv','w+') as t:
	t.write("IDEAL,PREDICT,TR,PR,PHIDEAL,PHICAUORIT,MULTINUM\n")
	for i in range(len(thetacc)):
		t.write(f"{THETALIST[i]},{thetacc[i]},{TR[i]},{PR[i]},{PHI[i]},{phc[i]},{phc[i]/(PHI[i])}\n")